<div style="background-color: #B7A6C9; padding:20px; border-radius:10px">
  <h1 style="color:#000000; text-align:center;">💰 Predicting Loan Payback 💰 </h1>
</div>

# 1. Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings("ignore")

# 2. Loading Data

In [ ]:
train_df=pd.read_csv("/kaggle/input/playground-series-s5e11/train.csv")
test_df=pd.read_csv("/kaggle/input/playground-series-s5e11/test.csv")
ss_df=pd.read_csv("/kaggle/input/playground-series-s5e11/sample_submission.csv")

In [ ]:
print("Training dataset shape :",train_df.shape)
print("Testing dataset shape :",test_df.shape)
print("Submission dataset shape :",ss_df.shape)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# 3. Initial Data Exploration

In [ ]:
#Show informations about the training dataset
train_df.info()

In [ ]:
#Display the statistical summary of the training dataset
train_df.describe()

In [ ]:
#Show informations about the testing dataset
test_df.info()

In [ ]:
#Display the statistical summary of the testing dataset
test_df.describe()

In [ ]:
train_df.columns

In [ ]:
train_df.drop(columns=["id"], inplace=True)

In [ ]:
#Selecting numeric columns in the training dataset
num_cols=train_df.select_dtypes(include=["number"]).columns.to_list()
print(num_cols)

In [ ]:
#Selecting categorical columns in the training dataset
cat_cols=train_df.select_dtypes(exclude=["number"]).columns.to_list()
print(cat_cols)

# 4. Checking Data Quality

In [ ]:
#Checking for missing values in the training dataset
train_df.isna().sum()

In [ ]:
#Checking for missing values in the testing datset
test_df.isna().sum()

In [ ]:
# Checking for duplicates in the training dataset
print("The number of duplicated observations in the train dataset is equal to", train_df.duplicated().sum())

# Checking for duplicates in the testing dataset
print("The number of duplicated observations in the test dataset is equal to", test_df.duplicated().sum())

# 5. Exploratory Data Analysis

In [ ]:
train_df.nunique().sort_values()

In [ ]:
sns.histplot(train_df['annual_income'],kde=True)
plt.show()

In [ ]:
sns.histplot(train_df['debt_to_income_ratio'],kde=True)
plt.show()

In [ ]:
sns.histplot(train_df['credit_score'],kde=True)
plt.show()

In [ ]:
sns.histplot(train_df['loan_amount'],kde=True)
plt.show()

In [ ]:
sns.histplot(train_df['interest_rate'],kde=True)
plt.show()

In [ ]:
sns.histplot(train_df['loan_paid_back'],kde=True)
plt.show()

In [ ]:
train_df["gender"].value_counts() #categorical value

In [ ]:
sns.countplot(data=train_df, x="gender")
plt.title("Gender Distribution")
plt.show()

In [ ]:
train_df["marital_status"].value_counts() #categorical value

In [ ]:
sns.countplot(data=train_df, x="marital_status")
plt.title("Distribution of Marital Status")
plt.show()

In [ ]:
train_df["education_level"].value_counts().sort_values() #categorical value

In [ ]:
sns.countplot(data=train_df, x="education_level", order=train_df["education_level"].value_counts().sort_values(ascending=False).index)
plt.title("Distribution of Education Levels")
plt.show()

In [ ]:
train_df["employment_status"].value_counts().sort_values() #categorical value

In [ ]:
sns.countplot(data=train_df, x="employment_status", order=train_df["employment_status"].value_counts().sort_values(ascending=False).index)
plt.title("Distribution of Employment Status")
plt.xticks(rotation=90)
plt.show()

In [ ]:
train_df["loan_purpose"].value_counts().sort_values() #categorical value

In [ ]:
sns.countplot(data=train_df, x="loan_purpose", order=train_df["loan_purpose"].value_counts().sort_values(ascending=False).index)
plt.title("Distribution of Loan Purposes")
plt.xticks(rotation=90)
plt.show()

In [ ]:
train_df["grade_subgrade"].value_counts()

In [ ]:
sns.countplot(data=train_df, x="grade_subgrade", order=train_df["grade_subgrade"].value_counts().sort_values(ascending=False).index)
plt.title("Distribution of Grade Subgrade")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(train_df[num_cols].corr(), annot=True, fmt=".2f")
plt.show()

# 6. Modeling


# 7. Hyperparameters Tuning & Optimisation

# 8. Submitting Predictions


In [ ]:
#y_pred=best_grad_model.predict(test_df.drop(columns=['id']))

In [ ]:
# submission = pd.DataFrame({'id': test_df['id'], 'BeatsPerMinute': y_pred})
# submission.to_csv('submission.csv', index=False)
# display(submission.head())